In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder



In [3]:
df_risco_train = pd.read_csv('risco_train.csv')


In [4]:
df_risco_test = pd.read_csv('risco_test.csv')

In [6]:
#df_risco_train.head()
#df_risco_test.head()

In [9]:
X = df_risco_train[['score_1','score_2','score_3','score_4','score_5', 'score_6', 'risk_rate', 'n_defaulted_loans']] 
y = df_risco_train.target_default
#X.score_1

In [10]:
lb = LabelEncoder()
X['score_1'] = lb.fit_transform(X.score_1)
X['score_2'] = lb.fit_transform(X.score_2)
X.head()

/anaconda2/envs/DataChallenge/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/envs/DataChallenge/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,n_defaulted_loans
0,0,10,350.0,101.800832,0.259555,108.427273,0.40,0.0
1,3,16,370.0,97.062615,0.942655,92.002546,0.24,0.0
2,3,9,360.0,100.027073,0.351918,112.892453,0.29,0.0
3,0,21,510.0,101.599485,0.987673,94.902491,0.32,0.0
4,2,1,500.0,98.474289,0.532539,118.126207,0.18,0.0


In [11]:
#Para otimizar o processameto diminuir a quantidade de registro 
X1 = np.asarray(X)
y1 = np.asarray(y)
X1 = X1[:10000,:]
y1 = y1[:10000]
#X1

# SVC
### Grid Search procurando os melhores parametros

In [8]:
# Make an fbeta_score scoring object.
scorer = make_scorer(accuracy_score)

In [112]:
parameters = {'kernel':['rbf'], 'C':[.1,.4], 'gamma':[.1,.3], 'degree':[1], 'random_state':[1, 2]}
parameters2 = {'kernel':['poly'], 'C':[.1, 1, 10], 'degree':[2,4,8] }


grid_obj = GridSearchCV(SVC(), parameters, scoring=scorer)


In [113]:
grid_fit = grid_obj.fit(X1, y1)

/anaconda2/envs/DataChallenge/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [114]:
#Get the estimator.
best = grid_fit.best_estimator_

In [115]:
best

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.001, verbose=False)

In [116]:
# Fit the new model.
#Agora treinar o modelo com todos registros 
best.fit(X, y)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.001, verbose=False)

In [117]:

# Make predictions using the new model.
best_train_predictions = best.predict(X)


In [123]:
acc = accuracy_score(y, best_train_predictions)

In [124]:
acc

0.84042068948995

### Usar direto os paramentros já encontrado no grid search
probability=True -> alterado para poder usar a funcao predict_proba

In [24]:
modelo_svc = SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=1, shrinking=True,
  tol=0.001, verbose=False)


In [26]:
modelo_svc = modelo_svc.fit(X,y)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=10, shrinking=True,
  tol=0.001, verbose=False) 
  ### 0.84042068948995

## Montando o arquivo test

In [133]:
df_risco_test = pd.read_csv('risco_test.csv')
sample_default = df_risco_test
#sample_default.head()

In [134]:
lb = LabelEncoder()
sample_default['score_1'] = lb.fit_transform(sample_default['score_1'])
sample_default['score_2'] = lb.fit_transform(sample_default['score_2'])

X_test = sample_default[['score_1','score_2','score_3','score_4','score_5', 'score_6', 'risk_rate', 'n_defaulted_loans']] 
X_test.head()

,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,n_defaulted_loans
0,7,30,710.0,104.174961,0.661509,123.015325,0.43,0.0
1,2,31,120.0,100.434557,0.139784,120.134718,0.15,0.0
2,4,10,280.0,95.399016,0.592517,101.661396,0.24,0.0
3,4,16,240.0,100.979281,0.588796,101.724127,0.20,0.0
4,3,35,430.0,101.287897,0.694936,114.951255,0.61,0.0


In [131]:
#separar os ids 
ids = sample_default['ids']
#usar o modelo validado para o df test
predictions = modelo_svc.predict_proba(X_test)


NameError: name 'sample_default' is not defined

In [39]:
predictions[:,0]

array([0.83512815, 0.83525702, 0.84136763, ..., 0.8451355 , 0.8489257 ,
       0.84718442])

In [41]:
#salvar o arquivo 
output = pd.DataFrame({ 'ids' : ids, 'default': predictions[:,0]})
output

,ids,default
0,810e3277-619e-3154-7ba0-ebddfc5f7ea9,0.835128
1,285ce334-3602-42b3-51cb-eebfcba48a09,0.835257
2,08efdf0f-f8f1-b968-3452-6b634eabf43f,0.841368
3,c81955ef-e3fe-e40e-b8b9-916dfea3b80d,0.839393
4,7f93590f-4a91-aec4-5062-cb1ba2c16a84,0.842059
5,ac81ce99-9365-b28e-31de-996d7aaadc95,0.856910
6,26a68732-a660-8cf9-340d-34c88ecdcbf0,0.835332
7,fd228aa8-5a67-6561-4382-21c35360c01d,0.848317
8,083ca221-7f4e-21c7-204d-2962c3ca9c14,0.848815
9,d040b1f4-9224-abf5-f489-b7c2055a8177,0.823511


In [42]:
#output.to_csv('default_submission.csv')

## Arvore de decisao 

In [5]:
from sklearn.tree import DecisionTreeClassifier


In [6]:
model = DecisionTreeClassifier()

In [122]:
parametros= {'max_depth':[20], 'min_samples_leaf': [20, 40], 'min_samples_split':[2,4],'random_state':[1,10,40,100]}
grid_obj_t = GridSearchCV(model, parametros, scoring=scorer)

In [123]:
best_gridt = grid_obj_t.fit(X,y)

/anaconda2/envs/DataChallenge/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [124]:
# TODO: Get the estimator.
best_t = best_gridt.best_estimator_

In [125]:
best_t

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=40, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best')

In [126]:
# TODO: Make predictions. Store them in the variable y_pred.
y_pred_t = best_t.predict(X)

In [127]:

# TODO: Calculate the accuracy and assign it to the variable acc.
accT = accuracy_score(y, y_pred_t)
accT

0.8427205864737308

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=40, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best')
            
## 0.8427205864737308

Como foi o modelo que mais teve acuracia foi o escolhido 


In [135]:
#separar os ids 
ids = sample_default['ids']
#usar o modelo validado para o df test
predictions_t = best_t.predict_proba(X_test)


In [138]:
#salvar o arquivo 
output = pd.DataFrame({ 'ids' : ids, 'default': predictions_t[:,0]})
output.head()

,ids,default
0,810e3277-619e-3154-7ba0-ebddfc5f7ea9,0.794521
1,285ce334-3602-42b3-51cb-eebfcba48a09,0.742424
2,08efdf0f-f8f1-b968-3452-6b634eabf43f,0.551724
3,c81955ef-e3fe-e40e-b8b9-916dfea3b80d,0.685185
4,7f93590f-4a91-aec4-5062-cb1ba2c16a84,0.804348


In [139]:
output.to_csv('default_submission.csv')

# Regressao Linear 

In [64]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.tree import DecisionTreeClassifier


In [61]:
model = LinearRegression()
model.fit(X, y)

/anaconda2/envs/DataChallenge/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [62]:
prediction = model.predict(X)

In [63]:
accL = accuracy_score(y1, y_pred_t)
accL

0.836

# Arvore de decisao